<a href="https://colab.research.google.com/github/bytebuster21/AI-project/blob/main/Train_the_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam # Explicitly import Adam optimizer
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger # Import useful callbacks
import matplotlib.pyplot as plt
import os
import shutil

# --- 1. Mount Google Drive (Not strictly required for dummy data, but kept for context) ---
# If you plan to save models/logs to Drive, or later switch to real data on Drive, run this.
from google.colab import drive
drive.mount('/content/drive')
print("Google Drive mounted successfully!")

# --- 2. Define dataset paths (using paths for dummy data) ---
# These paths will be used for the dummy dataset created below.
trainpath = '/content/train'
testpath = '/content/test'

print(f"Using dummy dataset located at: \n  Train: {trainpath}\n  Test: {testpath}")

# --- 3. Create Dummy Data for Demonstration ---
# This block creates placeholder images and directories, so the code runs out-of-the-box.
# This ensures a consistent environment for demonstration.
if os.path.exists(trainpath): # Clean up previous dummy data if it exists
    shutil.rmtree(trainpath)
if os.path.exists(testpath):
    shutil.rmtree(testpath)

print("Creating dummy dataset for demonstration...")
os.makedirs(os.path.join(trainpath, 'class_A'), exist_ok=True)
os.makedirs(os.path.join(trainpath, 'class_B'), exist_ok=True)
os.makedirs(os.path.join(trainpath, 'class_C'), exist_ok=True)

os.makedirs(os.path.join(testpath, 'class_A'), exist_ok=True)
os.makedirs(os.path.join(testpath, 'class_B'), exist_ok=True)
os.makedirs(os.path.join(testpath, 'class_C'), exist_ok=True)

from PIL import Image
def create_dummy_image(path, color, size=(299, 299)):
    img = Image.new('RGB', size, color=color)
    img.save(path)

# Create dummy training images (approx. 348 total to simulate previous outputs)
for i in range(116): # 116 * 3 classes = 348
    create_dummy_image(os.path.join(trainpath, 'class_A', f'imgA_{i:03d}.png'), (255, 100, 100))
    create_dummy_image(os.path.join(trainpath, 'class_B', f'imgB_{i:03d}.png'), (100, 255, 100))
    create_dummy_image(os.path.join(trainpath, 'class_C', f'imgC_{i:03d}.png'), (100, 100, 255))

# Create dummy testing images (approx. 153 total to simulate previous outputs)
for i in range(51): # 51 * 3 classes = 153
    create_dummy_image(os.path.join(testpath, 'class_A', f'testA_{i:03d}.png'), (200, 50, 50))
    create_dummy_image(os.path.join(testpath, 'class_B', f'testB_{i:03d}.png'), (50, 200, 50))
    create_dummy_image(os.path.join(testpath, 'class_C', f'testC_{i:03d}.png'), (50, 50, 200))

print(f"Dummy dataset created at {trainpath} and {testpath}")


# --- 4. Configure ImageDataGenerator instances ---
TARGET_SIZE = (299, 299) # VGG16 typically expects 224x224, but 299x299 is also supported.
BATCH_SIZE = 20

# Training Data Generator (with augmentation)
train_datagen = ImageDataGenerator(
    rescale=1./255,          # Normalize pixel values to [0, 1]
    rotation_range=20,       # Randomly rotate images by up to 20 degrees
    width_shift_range=0.1,   # Randomly shift images horizontally by up to 10%
    height_shift_range=0.1,  # Randomly shift images vertically by up to 10%
    shear_range=0.2,         # Apply shearing transformations
    zoom_range=0.2,          # Randomly zoom in on images
    horizontal_flip=True,    # Randomly flip images horizontally
    fill_mode='nearest'      # Strategy for filling newly created pixels
)

# Test Data Generator (ONLY rescaling, no augmentation)
# Augmentation is only for training data to prevent overfitting.
test_datagen = ImageDataGenerator(rescale=1./255)

# --- 5. Flow Images from Directories ---
# This loads images in batches and applies the transformations.
print("\nLoading images using ImageDataGenerator...")

train_generator = train_datagen.flow_from_directory(
    trainpath,
    target_size=TARGET_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical' # For multi-class classification (one-hot encoded labels)
)

test_generator = test_datagen.flow_from_directory(
    testpath,
    target_size=TARGET_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False # Essential for consistent evaluation results on test set
)

# Get number of classes and class names from the generator
num_classes = train_generator.num_classes
class_names = list(train_generator.class_indices.keys())

print(f"Found {train_generator.samples} training images belonging to {num_classes} classes.")
print(f"Found {test_generator.samples} test images belonging to {num_classes} classes.")
print("Class Names:", class_names)


# --- 6. Build the Model (VGG16 Feature Extractor + Custom Dense Layers) ---

# Load the VGG16 model pre-trained on ImageNet.
# `include_top=False` means we are excluding the original classification head
# as we will add our own custom layers for our specific dataset.
base_model = VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=(TARGET_SIZE[0], TARGET_SIZE[1], 3) # Input shape for our images
)

# Freeze the layers of the base model.
# This makes them non-trainable, preserving the learned ImageNet features.
# Only the newly added custom layers will be trained.
base_model.trainable = False

# Create a Sequential model and add the VGG16 base followed by custom classification layers.
model = Sequential([
    base_model, # The frozen VGG16 convolutional base
    Flatten(),  # Flattens the 3D output of the base model into a 1D vector
    Dense(256, activation='relu'), # First fully connected layer
    Dropout(0.5), # Dropout for regularization
    Dense(128, activation='relu'), # Second fully connected layer
    Dropout(0.4), # Another dropout layer
    Dense(num_classes, activation='softmax') # Output layer: one neuron per class, softmax for probabilities
])


# --- 7. Configure the Learning Process (model.compile()) ---

# Optimizer: Adam is a good general-purpose optimizer. Using a small learning rate.
chosen_optimizer = Adam(learning_rate=0.0001)

# Loss Function: 'categorical_crossentropy' for multi-class classification with one-hot encoded labels.
chosen_loss_function = 'categorical_crossentropy'

# Metrics: What to monitor during training (e.g., accuracy).
chosen_metrics = ['accuracy']

# Compile the model
model.compile(
    optimizer=chosen_optimizer,
    loss=chosen_loss_function,
    metrics=chosen_metrics
)

# Print the summary of the complete model architecture
print("\n--- Complete Model Summary (VGG16 with Custom Dense Layers) ---")
model.summary()


# --- 8. Define Callbacks for Training Management ---
# Callbacks are special functions called at various stages of the training process.

# Directory to save model checkpoints
checkpoint_dir = 'model_checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)
checkpoint_filepath = os.path.join(checkpoint_dir, 'best_model_vgg16.h5')

# 8.1 ModelCheckpoint: Save the best model encountered so far.
# Saves if the 'val_loss' (validation loss) is the least.
model_checkpoint = ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_loss',     # Monitor validation loss
    save_best_only=True,    # Only save if val_loss improves
    mode='min',             # 'min' mode for loss (we want the minimum loss)
    verbose=1               # Show messages when saving
)

# 7.2 EarlyStopping: Stop training if validation loss doesn't improve for 'patience' epochs.
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,             # Stop if no improvement for 5 epochs
    restore_best_weights=True, # Load weights from the best epoch before stopping
    verbose=1
)

# 7.3 ReduceLROnPlateau: Reduce learning rate if validation loss plateaus.
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,             # Reduce LR by factor of 0.2
    patience=3,             # Wait 3 epochs before reducing
    min_lr=0.000001,        # Minimum learning rate
    verbose=1
)

# 7.4 CSVLogger: Log training history to a CSV file.
csv_logger = CSVLogger('training_log.csv', append=True)

# List of all callbacks to be used during training
callbacks_list = [
    model_checkpoint,
    early_stopping,
    reduce_lr,
    csv_logger
]


# --- 9. Train the Model (model.fit()) ---
# This starts the training loop with your specified parameters and callbacks.
print("\n--- Starting Model Training ---")
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE, # Number of batches per epoch
    epochs=25, # Train for a maximum of 25 epochs
    validation_data=test_generator,
    validation_steps=test_generator.samples // BATCH_SIZE, # Number of validation batches per epoch
    callbacks=callbacks_list, # Apply all defined callbacks
    verbose=1 # Show training progress
)

print("\nModel training complete.")
print(f"Best model (based on validation loss) saved to: {checkpoint_filepath}")


# --- 10. Evaluate the Model ---
# After training, load the best saved model and evaluate its performance on the test set.
try:
    loaded_model = tf.keras.models.load_model(checkpoint_filepath)
    print(f"\nLoaded best model from {checkpoint_filepath}")
    loss, accuracy = loaded_model.evaluate(test_generator, steps=test_generator.samples // BATCH_SIZE)
    print(f"\nFinal Test Loss (best model): {loss:.4f}, Final Test Accuracy (best model): {accuracy:.4f}")
except Exception as e:
    print(f"\nCould not load best model for final evaluation: {e}")
    print("Evaluating the current model state (if training was not run or save failed).")
    loss, accuracy = model.evaluate(test_generator, steps=test_generator.samples // BATCH_SIZE)
    print(f"\nFinal Test Loss: {loss:.4f}, Final Test Accuracy: {accuracy:.4f}")


# --- 11. Plotting Training History ---
# Visualize the training and validation accuracy/loss over epochs.
if 'history' in locals() and history is not None:
    plt.figure(figsize=(12, 5))

    # Plot training & validation accuracy
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy over Epochs')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(loc='upper left')
    plt.grid(True)

    # Plot training & validation loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss over Epochs')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(loc='upper right')
    plt.grid(True)

    plt.tight_layout()
    plt.show()

# --- 12. Clean up dummy data and checkpoints (Optional) ---
# Uncomment these lines if you want to remove the dummy dataset and saved models after inspection.
if os.path.exists(trainpath):
    shutil.rmtree(trainpath)
if os.path.exists(testpath):
    shutil.rmtree(testpath)
if os.path.exists(checkpoint_dir):
    shutil.rmtree(checkpoint_dir)
if os.path.exists('training_log.csv'):
    os.remove('training_log.csv')
print(f"\nDummy dataset, checkpoints directory and training log file removed.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive mounted successfully!
Using dummy dataset located at: 
  Train: /content/train
  Test: /content/test
Creating dummy dataset for demonstration...
Dummy dataset created at /content/train and /content/test

Loading images using ImageDataGenerator...
Found 348 images belonging to 3 classes.
Found 153 images belonging to 3 classes.
Found 348 training images belonging to 3 classes.
Found 153 test images belonging to 3 classes.
Class Names: ['class_A', 'class_B', 'class_C']
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step

--- Complete Model Summary (VGG16 with Custom Dense Layers) ---


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ (None, 9, 9, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 41472)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │    10,617,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 25,365,059 (96.76 MB)

 Trainable params: 10,650,371 (40.63 MB)

...

17/17 ━━━━━━━━━━━━━━━━━━━━ 198s 11s/step - accuracy: 0.9000 - loss: 0.2314 - val_accuracy: 1.0000 - val_loss: 0.1300 - learning_rate: 1.0000e-04
Epoch 9/25
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 22s/step - accuracy: 0.9594 - loss: 0.2142 
Epoch 9: val_loss improved from 0.12999 to 0.06798, saving model to model_checkpoints/best_model_vgg16.h5


17/17 ━━━━━━━━━━━━━━━━━━━━ 560s 33s/step - accuracy: 0.9594 - loss: 0.2146 - val_accuracy: 1.0000 - val_loss: 0.0680 - learning_rate: 1.0000e-04
Epoch 10/25
 1/17 ━━━━━━━━━━━━━━━━━━━━ 5:36 21s/step - accuracy: 0.9500 - loss: 0.1369

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()



Epoch 10: val_loss improved from 0.06798 to 0.06583, saving model to model_checkpoints/best_model_vgg16.h5


17/17 ━━━━━━━━━━━━━━━━━━━━ 173s 9s/step - accuracy: 0.9500 - loss: 0.1369 - val_accuracy: 1.0000 - val_loss: 0.0658 - learning_rate: 1.0000e-04
Epoch 11/25
 1/17 ━━━━━━━━━━━━━━━━━━━━ 6:17 24s/step - accuracy: 1.0000 - loss: 0.1904